# Introduction

We will extract the relevant information after the algorithm analysed all the mammograms in the dataset.
Each patient has a total of 4 mammograms. We will take the algorithm final prediction as the most frequent result of the analysis of each mammographic view (the mode)

In [ ]:
import os
import fnmatch
import sys
import shutil
import re
import pandas as pd
import json

## Data cleaning from the results of the AI density tool

In [ ]:
#file resulting from the AI density analysis tool
file_csv = "demo.csv"
#list of process numbers extracted from the dataset
file_list_of_ids = "ids.json"
#dataframe resulting from the Radiologists density analysis
file_radiologists = "df_radiologist.json"
#dictionary of density values to map
density_values = {"1": "A", "2": "B", "3":"C", "4":"D"}

In [ ]:
f = open (file_list_of_ids, "r")
duplicated_list_of_ids = json.loads(f.read())
#Remove duplicated ids
list_of_ids = [*set(duplicated_list_of_ids)]

In [ ]:
df = pd.read_csv(file_csv,names=["process","aidensity"])

In [ ]:
df["process"] = df["process"].apply(lambda x: x.lstrip("""<_io.BufferedReader name='dataset/"""))
df["process"] = df["process"].apply(lambda x: x.rstrip(""".dcm'>"""))

In [ ]:
df['aidensity']=df['aidensity'].str.extract(r'(prediction":.)')
df["aidensity"] = df["aidensity"].apply(lambda x: x.lstrip("""prediction":"""))

## Extracting the results of all the mammographic views and creating the dataframe with the final ai density prediction

In [ ]:
df_to_transpose = pd.DataFrame([0,0])

In [ ]:
for i in list_of_ids:
    new_df = df.loc[df.process.str.contains(i, na=False)]
    mode_to_insert = new_df["aidensity"].mode()[0]
    df_to_transpose.insert(0, i, mode_to_insert, True)

In [ ]:
df_ai_densities = df_to_transpose.T
df_ai_densities.rename(columns = {0:'aidensity'}, inplace = True)
df_ai_densities.drop(df_ai_densities.index[-1], inplace=True)
df_ai_densities.drop(columns=1, inplace=True)
df_ai_densities.index.name = 'process'

In [ ]:
df_ai_densities = df_ai_densities.replace(density_values)

In [ ]:
print(f"A total of {df.shape[0]} unique mammograms were analised, resulting in the final density score for {df_ai_densities.shape[0]} unique patients.")

## Final AI density analysis of all the mammograms of the dataset
This data will be compared with the ground truth, to calculate the relevant metrics.

In [ ]:
df_ai_densities.to_json(r'df_ai_densities.json')